In [1]:
import json
import folium
from pyproj import Transformer

# Load the JSON data
with open('./data/충청북도_보은읍_이평리.json', encoding='utf-8') as f:
    data = json.load(f)

# Initialize the coordinate transformer (example: from EPSG:3857 to EPSG:4326)
transformer = Transformer.from_crs("EPSG:5179", "EPSG:4326")

# Extract the coordinates from the JSON data and convert them to lat/lon
coordinates_list = []
for item in data['output']['farmmapData']['data']:
    for geometry in item['geometry']:
        coordinates = [(transformer.transform(coord['y'], coord['x'])) for coord in geometry['xy']]
        coordinates_list.append(coordinates)

# Initialize a Folium map centered at an appropriate location
map_center = coordinates_list[0][0]
m = folium.Map(location=map_center, zoom_start=14)

# Add polygons to the map
for coordinates in coordinates_list:
    folium.Polygon(locations=coordinates, color='blue', weight=2, fill=True, fill_color='blue').add_to(m)

# Save the map to an HTML file
m.save('map.html')


In [2]:
m

In [4]:
# add popup

import json
import folium
from pyproj import Transformer

# Load the JSON data
with open('./data/충청북도_보은읍_이평리.json', encoding='utf-8') as f:
    data = json.load(f)

# Initialize the coordinate transformer (example: from EPSG:5179 to EPSG:4326)
transformer = Transformer.from_crs("EPSG:5179", "EPSG:4326")

# Extract the coordinates from the JSON data and convert them to lat/lon
coordinates_list = []
for item in data['output']['farmmapData']['data']:
    polygon_info = {
        'uid': item['uid'],
        'pnu': item['pnu'],
        'coordinates': []
    }
    for geometry in item['geometry']:
        coordinates = [(transformer.transform(coord['y'], coord['x'])) for coord in geometry['xy']]
        polygon_info['coordinates'].append(coordinates)
    coordinates_list.append(polygon_info)

# Initialize a Folium map centered at an appropriate location
map_center = coordinates_list[0]['coordinates'][0][0]
m = folium.Map(location=map_center, zoom_start=14)

# Add polygons to the map with popup info
for polygon_info in coordinates_list:
    for coordinates in polygon_info['coordinates']:
        folium.Polygon(
            locations=coordinates, 
            color='yellow', 
            weight=2, 
            fill=True, 
            fill_color='yellow',
            popup=f"UID: {polygon_info['uid']}<br>PNU: {polygon_info['pnu']}"
        ).add_to(m)

# Save the map to an HTML file
m.save('map.html')


In [5]:
m

In [6]:
# read multiple file and add popup

import json
import folium
from pyproj import Transformer

# List of file paths and corresponding colors
files = [
    {'path': './data/전라남도_나주시_노안면_학산리_논.json', 'color': 'yellow'},
    {'path': './data/전라남도_나주시_노안면_학산리_밭.json', 'color': 'green'},
    {'path': './data/전라남도_나주시_노안면_학산리_과수.json', 'color': 'red'},
    {'path': './data/전라남도_나주시_노안면_학산리_비경지.json', 'color': 'brown'},
    {'path': './data/전라남도_나주시_노안면_학산리_시설.json', 'color': 'gray'}
]

# Initialize the coordinate transformer (example: from EPSG:5179 to EPSG:4326)
transformer = Transformer.from_crs("EPSG:5179", "EPSG:4326")

# Initialize a Folium map centered at an appropriate location (initially unknown, will be updated later)
map_center = [0, 0]
m = folium.Map(location=map_center, zoom_start=14)

# Flag to check if the map center is set
center_set = False

# Process each file
for file_info in files:
    with open(file_info['path'], encoding='utf-8') as f:
        data = json.load(f)

    # Extract the coordinates from the JSON data and convert them to lat/lon
    coordinates_list = []
    for item in data['output']['farmmapData']['data']:
        polygon_info = {
            'uid': item['uid'],
            'pnu': item['pnu'],
            'coordinates': []
        }
        for geometry in item['geometry']:
            coordinates = [(transformer.transform(coord['y'], coord['x'])) for coord in geometry['xy']]
            polygon_info['coordinates'].append(coordinates)
        coordinates_list.append(polygon_info)

    # Set the map center to the first coordinate of the first file if not already set
    if not center_set:
        map_center = coordinates_list[0]['coordinates'][0][0]
        m.location = map_center
        center_set = True

    # Add polygons to the map with popup info and different colors
    for polygon_info in coordinates_list:
        for coordinates in polygon_info['coordinates']:
            folium.Polygon(
                locations=coordinates, 
                color=file_info['color'], 
                weight=2, 
                fill=True, 
                fill_color=file_info['color'],
                popup=f"UID: {polygon_info['uid']}<br>PNU: {polygon_info['pnu']}"
            ).add_to(m)

# Save the map to an HTML file
m.save('map.html')


In [7]:
m